In [ ]:
def diagnosis_assistant():
    st.title("Diagnostic Assistant 🩺")
    
    # Initialize chat history if not already present
    if "chat_history" not in st.session_state:
        st.session_state.chat_history = [{"role": "assistant", "content": INITIAL_RESPONSE}]
    
    # Display previous chat messages
    for message in st.session_state.chat_history:
        role = "user" if message["role"] == "user" else "assistant"
        avatar = "🗨️" if role == "user" else "💉"
        with st.chat_message(role, avatar=avatar):
            st.markdown(message["content"])
    
    # Get new input from the user
    user_prompt = st.chat_input("Describe your symptoms here...")
    if user_prompt:
        with st.chat_message("user", avatar="🗨️"):
            st.markdown(user_prompt)
        st.session_state.chat_history.append({"role": "user", "content": user_prompt})
        
        # Create the conversation messages with context
        messages = [
            {"role": "system", "content": CHAT_CONTEXT},
            {"role": "assistant", "content": INITIAL_RESPONSE}
        ] + st.session_state.chat_history
        
        # Get response from the Groq API using the LLM
        with st.chat_message("assistant", avatar="💉"):
            stream = client.chat.completions.create(
                model="llama3-8b-8192",
                messages=messages,
                stream=True
            )
            response_content = "".join([chunk.choices[0].delta.content or "" for chunk in stream])
            st.markdown(response_content)
        
        # Append the assistant's response to the chat history
        st.session_state.chat_history.append({"role": "assistant", "content": response_content})
